# Drag Coff. prediction using Pointnet 

## Pre processing

### Mesh -> Point Cloud

In [72]:
import os
import glob
import numpy as np
import open3d as o3d
from tqdm import tqdm

def mesh_to_pointcloud(stl_path, n_samples=4056):
    """
    Converts an STL mesh to a normalized point cloud using 
    Surface Sampling followed by Farthest Point Sampling (FPS).
    """
    # 1. Load Mesh
    mesh = o3d.io.read_triangle_mesh(stl_path)
    
    # 2. Pre-process Mesh (Essential for DrivAerNet++ STLs)
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_vertices()
    
    # 3. Surface Sampling 
    # We sample 5x more points than needed initially to give FPS a good 'pool' to pick from
    temp_pcd = mesh.sample_points_uniformly(number_of_points=n_samples * 5)
    pts = np.asarray(temp_pcd.points).astype(np.float32)
    
    # 4. Normalize (Center and Scale to Unit Sphere)
    centroid = np.mean(pts, axis=0)
    pts -= centroid
    scale = np.max(np.linalg.norm(pts, axis=1))
    pts /= (scale + 1e-9)
    
    # 5. Farthest Point Sampling (FPS) 
    # This ensures the 2048 points are spread perfectly over the car's geometry
    fps_indices = farthest_point_sample_numpy(pts, n_samples)
    final_pts = pts[fps_indices]
    
    return final_pts, centroid, scale

def farthest_point_sample_numpy(points, n_samples):
    """ Standard FPS implementation """
    N = points.shape[0]
    centroids = np.zeros((n_samples,), dtype=np.int64)
    distances = np.ones((N,), dtype=np.float32) * 1e10
    farthest = np.random.randint(0, N)

    for i in range(n_samples):
        centroids[i] = farthest
        centroid = points[farthest]
        dist = np.sum((points - centroid) ** 2, axis=1)
        mask = dist < distances
        distances[mask] = dist[mask]
        farthest = np.argmax(distances)
    return centroids

def process_dataset(input_dir, output_dir, n_points=2048):
    os.makedirs(output_dir, exist_ok=True)
    stl_files = sorted(glob.glob(os.path.join(input_dir, "*.stl")))
    
    print(f"Found {len(stl_files)} STL files. Starting conversion...")
    
    for stl_path in tqdm(stl_files):
        fname = os.path.splitext(os.path.basename(stl_path))[0]
        out_path = os.path.join(output_dir, f"{fname}.npz")
        
        if os.path.exists(out_path): continue
        
        try:
            pts, center, scale = mesh_to_pointcloud(stl_path, n_samples=n_points)
            
            # Save as compressed numpy for PointNet training
            np.savez(out_path, points=pts, centroid=center, scale=scale)
        except Exception as e:
            print(f"Error processing {fname}: {e}")

if __name__ == "__main__":
    # Update these paths to your DrivAerNet++ folders
    INPUT_FOLDER = "02_dataset/mesh_new/F_D_WM_WW_1_subset"
    OUTPUT_FOLDER = "02_dataset/car_4096_pc/pc_processed_2"
    
    process_dataset(INPUT_FOLDER, OUTPUT_FOLDER, n_points=4096)

Found 200 STL files. Starting conversion...


100%|██████████| 200/200 [11:04<00:00,  3.32s/it]


### Visualize the processed point cloud

In [2]:
import numpy as np
import open3d as o3d
import os

def npz_to_ply(npz_path, output_ply_path, denormalize=True):
    """
    Loads a processed .npz file and saves it as a .ply point cloud.
    
    Args:
        npz_path (str): Path to the input .npz file.
        output_ply_path (str): Path where the .ply will be saved.
        denormalize (bool): If True, uses saved centroid and scale to 
                           restore original car dimensions.
    """
    # 1. Load the NPZ data
    data = np.load(npz_path)
    pts = data['points']      # Shape: (N, 3)
    
    # 2. Denormalize (Optional)
    # This moves the car back from the "unit sphere" to its original CAD coordinates
    if denormalize and 'centroid' in data and 'scale' in data:
        centroid = data['centroid']
        scale = data['scale']
        pts = (pts * scale) + centroid
        print(f"Denormalized using scale: {scale:.4f}")

    # 3. Create Open3D PointCloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)

    # 4. Save to PLY
    o3d.io.write_point_cloud(output_ply_path, pcd)
    print(f"Successfully saved PLY to: {output_ply_path}")

# --- Example Usage ---
if __name__ == "__main__":
    # Change these to your actual paths
    INPUT_NPZ = "/home/exouser/01_project/01_AE_triangular_mesh/02_dataset/car_design_mesh/pc_processed_2/F_D_WM_WW_0101.npz"
    OUTPUT_PLY = "pc_car_4098_denormalised.ply"

    if os.path.exists(INPUT_NPZ):
        npz_to_ply(INPUT_NPZ, OUTPUT_PLY, denormalize=False)
    else:
        print(f"File {INPUT_NPZ} not found.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
File /home/exouser/01_project/01_AE_triangular_mesh/02_dataset/car_design_mesh/pc_processed_2/F_D_WM_WW_0101.npz not found.


### Labels Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json

In [18]:
RAW_CSV_PATH = "02_dataset/cd_data_303/DrivAerNetPlusPlus_Cd_303.csv"   # ID, Drag_Value

df = pd.read_csv(RAW_CSV_PATH)
print(df.head())
print(f"Total samples: {len(df)}")


               ID  Drag_Value
0  F_D_WM_WW_0001    0.275833
1  F_D_WM_WW_0002    0.279602
2  F_D_WM_WW_0003    0.272837
3  F_D_WM_WW_0004    0.257712
4  F_D_WM_WW_0005    0.267890
Total samples: 300


In [19]:
TRAIN_RATIO = 0.7
VAL_RATIO   = 0.15
TEST_RATIO  = 0.15

assert abs(TRAIN_RATIO + VAL_RATIO + TEST_RATIO - 1.0) < 1e-6


In [20]:
# First split: Train vs (Val + Test)
train_df, temp_df = train_test_split(
    df,
    test_size=(1.0 - TRAIN_RATIO),
    random_state=42,
    shuffle=True
)

# Second split: Val vs Test
val_df, test_df = train_test_split(
    temp_df,
    test_size=TEST_RATIO / (VAL_RATIO + TEST_RATIO),
    random_state=42,
    shuffle=True
)

print(f"Train samples: {len(train_df)}")
print(f"Val samples:   {len(val_df)}")
print(f"Test samples:  {len(test_df)}")


Train samples: 209
Val samples:   45
Test samples:  46


In [21]:
cd_min = train_df["Drag_Value"].min()
cd_max = train_df["Drag_Value"].max()

print("Cd_min (train):", cd_min)
print("Cd_max (train):", cd_max)


Cd_min (train): 0.2406969401668661
Cd_max (train): 0.3168595800557837


In [23]:
norm_metadata = {
    "Cd_min": float(cd_min),
    "Cd_max": float(cd_max)
}

with open("02_dataset/cd_data_303/drag_normalization.json", "w") as f:
    json.dump(norm_metadata, f, indent=4)

print("Saved normalization metadata → drag_normalization.json")


Saved normalization metadata → drag_normalization.json


In [24]:
def min_max_normalize(cd, cd_min, cd_max):
    return (cd - cd_min) / (cd_max - cd_min)
for split_df in [train_df, val_df, test_df]:
    split_df["Drag_Value_Norm"] = min_max_normalize(
        split_df["Drag_Value"],
        cd_min,
        cd_max
    )

    print("Train normalized range:",
      train_df["Drag_Value_Norm"].min(),
      train_df["Drag_Value_Norm"].max())



Train normalized range: 0.0 1.0
Train normalized range: 0.0 1.0
Train normalized range: 0.0 1.0


In [25]:
train_out = train_df[["ID", "Drag_Value_Norm"]]
val_out   = val_df[["ID", "Drag_Value_Norm"]]
test_out  = test_df[["ID", "Drag_Value_Norm"]]


In [28]:
train_out.to_csv("02_dataset/cd_data_303/drag_train_normalized.csv", index=False)
val_out.to_csv("02_dataset/cd_data_303/drag_val_normalized.csv", index=False)
test_out.to_csv("02_dataset/cd_data_303/drag_test_normalized.csv", index=False)

print("Saved:")
print("- drag_train_normalized.csv")
print("- drag_val_normalized.csv")
print("- drag_test_normalized.csv")


Saved:
- drag_train_normalized.csv
- drag_val_normalized.csv
- drag_test_normalized.csv


## PointNet:
- Input : Point Cloud. 
- 6 Convolution layers. 
- Channel increase: 64, 128, 256, 256, 512 Upto embedding dim (1024).
- Batch Normalisation.
- Residual connection link input -> Aiding gradient flow.
- 3 Linear layers reduce: 1024 -> 512. 
- Finally single output for drag.

## Data Loading and Dataset Class

### Import Libraries

In [15]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


### Dataset Class

In [29]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader


In [35]:
class DrivAerPointNetDataset(Dataset):
    def __init__(self, csv_file, pointcloud_dir):
        """
        Args:
            csv_file (str): Path to normalized CSV (train/val/test)
            pointcloud_dir (str): Directory containing .npz point clouds
        """
        self.data = pd.read_csv(csv_file)
        self.pointcloud_dir = pointcloud_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        sample_id = row["ID"]
        cd = row["Drag_Value_Norm"]  # already normalized

        pc_path = os.path.join(self.pointcloud_dir, f"{sample_id}.npz")
        pc_data = np.load(pc_path)

        points = pc_data["points"]  # (N, 3)

        # Convert to torch tensors
        points = torch.from_numpy(points).float()      # (N, 3)
        cd = torch.tensor(cd).float()                   # scalar

        return points, cd


In [36]:
POINTCLOUD_DIR = "02_dataset/car_4096_pc/pc_processed"
CSV_DIR = "02_dataset/cd_data_303"

train_dataset = DrivAerPointNetDataset(
    csv_file=os.path.join(CSV_DIR, "drag_train_normalized.csv"),
    pointcloud_dir=POINTCLOUD_DIR
)

val_dataset = DrivAerPointNetDataset(
    csv_file=os.path.join(CSV_DIR, "drag_val_normalized.csv"),
    pointcloud_dir=POINTCLOUD_DIR
)

test_dataset = DrivAerPointNetDataset(
    csv_file=os.path.join(CSV_DIR, "drag_test_normalized.csv"),
    pointcloud_dir=POINTCLOUD_DIR
)


In [37]:
BATCH_SIZE = 8   # adjust based on GPU memory
NUM_WORKERS = 4  # set 0 if debugging

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)


In [38]:
points, cd = next(iter(train_loader))

print("Points shape:", points.shape)  # (B, N, 3)
print("Cd shape:", cd.shape)          # (B,)
print("Cd range:", cd.min().item(), cd.max().item())


Points shape: torch.Size([8, 4096, 3])
Cd shape: torch.Size([8])
Cd range: 0.2213650643825531 0.7796438336372375


## Model Architecture

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [40]:
class PointNetBackbone(nn.Module):
    def __init__(self, emb_dim=1024):
        super().__init__()
        # Conv layers
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 256, 1)
        self.conv4 = nn.Conv1d(256, 256, 1)
        self.conv5 = nn.Conv1d(256, 512, 1)
        self.conv6 = nn.Conv1d(512, emb_dim, 1)
        
        # BatchNorm layers
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(256)
        self.bn5 = nn.BatchNorm1d(512)
        self.bn6 = nn.BatchNorm1d(emb_dim)
        
    def forward(self, x):
        # x: (B, N, 3)
        x = x.transpose(2, 1)  # (B, 3, N)

        # Conv + BN + ReLU
        x1 = F.relu(self.bn1(self.conv1(x)))  # 64 channels
        x2 = F.relu(self.bn2(self.conv2(x1))) # 128
        x3 = F.relu(self.bn3(self.conv3(x2))) # 256
        x4 = F.relu(self.bn4(self.conv4(x3))) # 256
        x5 = F.relu(self.bn5(self.conv5(x4))) # 512
        x6 = self.bn6(self.conv6(x5))         # 1024, no ReLU here (residual will be added)
        
        return x1, x6  # return early feat for residual + deep embedding


In [41]:
class PointNetRegressor(nn.Module):
    def __init__(self, emb_dim=1024):
        super().__init__()
        self.backbone = PointNetBackbone(emb_dim=emb_dim)
        
        # Residual projection: 64 → 1024
        self.res_proj = nn.Conv1d(64, emb_dim, 1)
        
        # Fully connected regression head
        self.fc1 = nn.Linear(emb_dim, 512)
        self.bn_fc1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 1)
        
    def forward(self, x):
        # x: (B, N, 3)
        early_feat, deep_feat = self.backbone(x)  # x1, x6
        
        # Residual connection
        res = self.res_proj(early_feat)
        feat = deep_feat + res
        
        # Global max pooling
        global_feat, _ = torch.max(feat, 2)  # (B, 1024)
        
        # FC regression
        x = F.relu(self.bn_fc1(self.fc1(global_feat)))
        x = self.fc2(x)
        
        return x.squeeze(1)  # (B,)


In [42]:
model = PointNetRegressor(emb_dim=1024)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

# Grab one batch
points, cd = next(iter(train_loader))
points = points.to(model.fc1.weight.device)

# Forward pass
output = model(points)
print("Output shape:", output.shape)  # Expected: (B,)
print("Sample outputs:", output[:5])


Output shape: torch.Size([8])
Sample outputs: tensor([ 0.7969,  0.0417,  0.6766, -0.2740,  0.4204], device='cuda:0',
       grad_fn=<SliceBackward0>)


## Training Loop

In [43]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


In [44]:
# Paths
CHECKPOINT_DIR = "checkpoints/pc_dc_pn"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

LOG_FILE = "logs/training_log.txt"

In [51]:
# Training settings
NUM_EPOCHS = 500
BATCH_SIZE = 8
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SAVE_EVERY = 10  # save checkpoint every 10 epochs


In [52]:
# Model
model = PointNetRegressor(emb_dim=1024).to(DEVICE)

# Loss
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LR)


In [53]:
start_epoch = 0
checkpoint_files = sorted([f for f in os.listdir(CHECKPOINT_DIR) if f.endswith(".pth")])

if checkpoint_files:
    latest_ckpt = os.path.join(CHECKPOINT_DIR, checkpoint_files[-1])
    print(f"Resuming from checkpoint: {latest_ckpt}")
    
    ckpt = torch.load(latest_ckpt, map_location=DEVICE)
    model.load_state_dict(ckpt["model_state_dict"])
    optimizer.load_state_dict(ckpt["optimizer_state_dict"])
    start_epoch = ckpt["epoch"] + 1
else:
    print("No checkpoint found, starting from scratch.")


Resuming from checkpoint: checkpoints/pc_dc_pn/checkpoint_epoch_90.pth


In [54]:
for epoch in range(start_epoch, NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    
    for points, cd in train_loader:
        points = points.to(DEVICE)
        cd = cd.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, cd)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * points.size(0)
    
    train_loss = running_loss / len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss_total = 0.0
    with torch.no_grad():
        for points, cd in val_loader:
            points = points.to(DEVICE)
            cd = cd.to(DEVICE)
            
            outputs = model(points)
            val_loss_total += criterion(outputs, cd).item() * points.size(0)
    
    val_loss = val_loss_total / len(val_loader.dataset)
    
    # Logging
    log_line = f"Epoch [{epoch+1}/{NUM_EPOCHS}]  Train Loss: {train_loss:.6f}  Val Loss: {val_loss:.6f}"
    print(log_line)
    
    with open(LOG_FILE, "a") as f:
        f.write(log_line + "\n")
    
    # Save checkpoint
    if (epoch + 1) % SAVE_EVERY == 0 or (epoch + 1) == NUM_EPOCHS:
        ckpt_path = os.path.join(CHECKPOINT_DIR, f"checkpoint_epoch_{epoch+1}.pth")
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }, ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")


Epoch [91/500]  Train Loss: 0.015663  Val Loss: 0.872477
Epoch [92/500]  Train Loss: 0.018823  Val Loss: 0.269228
Epoch [93/500]  Train Loss: 0.011082  Val Loss: 0.303799
Epoch [94/500]  Train Loss: 0.019216  Val Loss: 1.271782
Epoch [95/500]  Train Loss: 0.019308  Val Loss: 0.056256
Epoch [96/500]  Train Loss: 0.016813  Val Loss: 2.154948
Epoch [97/500]  Train Loss: 0.016059  Val Loss: 0.227945
Epoch [98/500]  Train Loss: 0.014917  Val Loss: 0.139216
Epoch [99/500]  Train Loss: 0.012038  Val Loss: 0.060865
Epoch [100/500]  Train Loss: 0.009778  Val Loss: 0.045446
Saved checkpoint: checkpoints/pc_dc_pn/checkpoint_epoch_100.pth
Epoch [101/500]  Train Loss: 0.012941  Val Loss: 0.404086
Epoch [102/500]  Train Loss: 0.016186  Val Loss: 0.112257
Epoch [103/500]  Train Loss: 0.012915  Val Loss: 0.430439
Epoch [104/500]  Train Loss: 0.018836  Val Loss: 0.237765
Epoch [105/500]  Train Loss: 0.015643  Val Loss: 0.138575
Epoch [106/500]  Train Loss: 0.011903  Val Loss: 0.365458
Epoch [107/500]  

In [55]:
import torch
import os

MODEL_DIR = "saved_models"
os.makedirs(MODEL_DIR, exist_ok=True)

# Save the model weights
MODEL_PATH = os.path.join(MODEL_DIR, "pointnet_cd_regressor.pth")
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to: {MODEL_PATH}")


Model saved to: saved_models/pointnet_cd_regressor.pth


## Inference: 

In [57]:
import json
# This contains the min/max Cd from the training set
with open("02_dataset/cd_data_303/drag_normalization.json") as f:
    norm_metadata = json.load(f)

CD_MIN = norm_metadata["Cd_min"]
CD_MAX = norm_metadata["Cd_max"]

def inverse_normalize(cd_norm):
    return cd_norm * (CD_MAX - CD_MIN) + CD_MIN


In [ ]:
model.eval()  # Important!
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(DEVICE)

In [ ]:
all_preds = []
all_true = []

with torch.no_grad():
    for points, cd in test_loader:
        points = points.to(DEVICE)
        cd = cd.to(DEVICE)
        
        outputs = model(points)  # normalized Cd
        all_preds.append(outputs.cpu())
        all_true.append(cd.cpu())

In [ ]:
all_preds = torch.cat(all_preds, dim=0).numpy()
all_true = torch.cat(all_true, dim=0).numpy()

In [ ]:
all_preds_phys = inverse_normalize(all_preds)
all_true_phys = inverse_normalize(all_true)

In [ ]:
import numpy as np

diff = all_preds_phys - all_true_phys
mae = np.mean(np.abs(diff))
rmse = np.sqrt(np.mean(diff ** 2))

print(f"Test MAE (Cd): {mae:.6f}")
print(f"Test RMSE (Cd): {rmse:.6f}")

Test MAE (Cd): 0.064325
Test RMSE (Cd): 0.066412
